# 0. Load MIMIC Data

In [164]:
import sys; print(sys.executable); print(sys.path)

"""
https://github.com/jupyter/notebook/issues/2359#issuecomment-318056024
If having installation issues, try

[path to python] -m pip install [package]

where [path to python] is the output of sys.executable.

e.g.
/opt/conda/envs/opennotes/bin/python -m pip install scispacy==0.2.5 spacy==2.3.1 nltk

Issues with spacy & scispacy versions

https://stackoverflow.com/questions/66149878/e053-could-not-read-config-cfg-resumeparser
# !pip install spacy==2.3.1
# !pip install scispacy==0.2.5
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_sm-0.2.5.tar.gz
"""

/opt/conda/envs/opennotes/bin/python
['/home/yutsumi', '/opt/conda/envs/opennotes/lib/python37.zip', '/opt/conda/envs/opennotes/lib/python3.7', '/opt/conda/envs/opennotes/lib/python3.7/lib-dynload', '', '/opt/conda/envs/opennotes/lib/python3.7/site-packages', '/opt/conda/envs/opennotes/lib/python3.7/site-packages/IPython/extensions', '/home/yutsumi/.ipython']


'\nhttps://github.com/jupyter/notebook/issues/2359#issuecomment-318056024\nIf having installation issues, try\n\n[path to python] -m pip install [package]\n\nwhere [path to python] is the output of sys.executable.\n\ne.g.\n/opt/conda/envs/opennotes/bin/python -m pip install scispacy==0.2.5 spacy==2.3.1 nltk\n\nIssues with spacy & scispacy versions\n\nhttps://stackoverflow.com/questions/66149878/e053-could-not-read-config-cfg-resumeparser\n# !pip install spacy==2.3.1\n# !pip install scispacy==0.2.5\n# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_sm-0.2.5.tar.gz\n'

In [165]:
# Import libraries
import numpy as np
import pandas as pd

import os 
import random
from collections import Counter

### Load tokenized MIMIC

In [166]:
note_dir = 'Tokenize_Toy'
# note_file = 'Copy of note_100166_174_sents.csv'
note_files = os.listdir(note_dir)

all_note_strs = []
all_sent_strs = []
for note_file in note_files:
    note_filename = os.path.join(note_dir, note_file)
    note_df = pd.read_csv(note_filename)
    
    proc_note_values = list(map(lambda x: x.replace('\n', ' '), \
                                note_df['0'].values))
    note_str = '. '.join(proc_note_values)
    
    all_note_strs.append(note_str)
    all_sent_strs.extend(proc_note_values)

In [167]:
note_str

'Admission Date: [**2119-5-4**]. Discharge Date: [**2119-5-25**]. Service: CARDIOTHORACIC. Allergies: Amlodipine. Attending: [**Last Name (NamePattern1) 1561**]. Chief Complaint:. 81 yo F smoker w/ COPD, severe TBM, s/p tracheobronchoplasty  [**5-5**]  s/p perc trach  [**5-13**]. Major Surgical or Invasive Procedure:. bronchoscopy 3/31,4/2,3, [**6-12**] ,  [**5-17**] ,  [**5-19**]  s/p trachealplasty  [**5-5**]  percutaneous tracheostomy  [**5-13**]  after failed extubation down size trach on  [**5-25**]  to size 6 cuffless. History of Present Illness:. years she has had progressive difficulties with her breathing.. In  [**2118-6-4**]  she was admitted to  [**Hospital1 18**]  for respiratory failure due to a COPD exacerbation.. Due to persistent hypoxemia, she required intubation and a eventual bronchoscopy on  [**2118-6-9**]  revealed marked. narrowing of the airways on expiration consistent with tracheomalacia.. She subsequently underwent placement of two silicone stents, one in the 

In [168]:
all_sent_strs[314]

'1. hydroxychloroquine 200 mg Tablet Sig: One (1) Tablet PO DAILY (Daily).'

# 1. Tools for Data Processing/Filtering

## [MetaMap](https://metamap.nlm.nih.gov/)

Tool to recognize UMLS concepts in text. Slow and bad.

Python wrapper: https://github.com/AnthonyMRios/pymetamap

## [Scispacy](https://allenai.github.io/scispacy/)

Python package containing spaCy models for processing biomedical, scientific or clinical text.

In [169]:
!python --version

Python 3.7.10


### Installations

In [170]:
# Should be installed in conda env
# !pip install spacy==2.3.1
# !pip install scispacy==0.2.5
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_sm-0.2.5.tar.gz

### Test some examples

In [171]:
import sys; print(sys.executable)

/opt/conda/envs/opennotes/bin/python


In [172]:
import spacy
import scispacy

from pprint import pprint
from collections import OrderedDict

from spacy import displacy
from scispacy.abbreviation import AbbreviationDetector

In [173]:
nlp = spacy.load("en_core_sci_sm")

#### Entity Display

In [12]:
# Source: https://github.com/WuraolaOyewusi/How-to-use-ScispaCy-for-Biomedical-Named-Entity-Recognition-Abbreviation-Resolution-and-link-UMLS/blob/master/Explore_ScispaCy.ipynb

def display_entities(nlp,document):
    """ 
    This function displays word entities

    Parameters: 
         model(module): A pretrained model from spaCy(https://spacy.io/models) or ScispaCy(https://allenai.github.io/scispacy/)
         document(str): Document to be processed

    Returns: Image rendering and list of named/unnamed word entities and entity labels 
     """
    # nlp = model.load()
    doc = nlp(document)
    displacy_image = displacy.render(doc, jupyter=True,style='ent')
    entity_and_label = pprint(set([(X.text, X.label_) for X in doc.ents]))
    return  displacy_image, entity_and_label


In [13]:
displacy_image, entity_and_label = display_entities(nlp, note_str)

{('**', 'ENTITY'),
 ('**2151', 'ENTITY'),
 ('**Hospital 1807', 'ENTITY'),
 ('**Last Name', 'ENTITY'),
 ('ADDENDUM', 'ENTITY'),
 ('Abdominal CT', 'ENTITY'),
 ('Admission', 'ENTITY'),
 ('CT', 'ENTITY'),
 ('HEAD CT', 'ENTITY'),
 ('Head CT', 'ENTITY'),
 ('JOB', 'ENTITY'),
 ('M.D', 'ENTITY'),
 ('MEDQUIST36', 'ENTITY'),
 ('Name8', 'ENTITY'),
 ('NamePattern1', 'ENTITY'),
 ('NamePattern2', 'ENTITY'),
 ('Number(1', 'ENTITY'),
 ('RADIOLOGIC', 'ENTITY'),
 ('Radiologic studies', 'ENTITY'),
 ('STUDIES', 'ENTITY'),
 ('T10', 'ENTITY'),
 ('cavitary lesions', 'ENTITY'),
 ('consistent with', 'ENTITY'),
 ('imaging', 'ENTITY'),
 ('infarction', 'ENTITY'),
 ('infectious process/tuberculosis', 'ENTITY'),
 ('intracranial hemorrhage', 'ENTITY'),
 ('left lung apex', 'ENTITY'),
 ('lesions', 'ENTITY'),
 ('mass effect', 'ENTITY'),
 ('medical history', 'ENTITY'),
 ('moderate-sized left pleural effusion', 'ENTITY'),
 ('osteoporosis', 'ENTITY'),
 ('outpatient', 'ENTITY'),
 ('sacrum', 'ENTITY')}


#### Medical Abbreviation

In [65]:
def show_medical_abbreviation(nlp,document):
    """ 
    This function detects and resolves medical abbreviations in word entities

    Parameters: 
         model(module): A pretrained biomedical model from ScispaCy(https://allenai.github.io/scispacy/)
         document(str): Document to be processed

    Returns: List of unique abbreviations and their resolution 
     """
    # nlp = model.load()
    abbreviation_pipe = AbbreviationDetector(nlp)
    try:
        nlp.add_pipe(abbreviation_pipe)
    except ValueError:
        pass
    doc = nlp(document)
    abbreviated = list(set([f"{abrv}  {abrv._.long_form}" for abrv in doc._.abbreviations]))             #list is set to ensure only unique values are returned
    return abbreviated

In [66]:
note_str

'Admission Date: [**2151-7-16**]. Discharge Date: [**2151-8-4**]. Service:. ADDENDUM:. RADIOLOGIC STUDIES:. Radiologic studies also included a chest CT, which confirmed cavitary lesions in the left lung apex consistent with infectious process/tuberculosis.. This also moderate-sized left pleural effusion.. HEAD CT:. Head CT showed no intracranial hemorrhage or mass effect, but old infarction consistent with past medical history.. ABDOMINAL CT:. Abdominal CT showed lesions of T10 and sacrum most likely secondary to osteoporosis.. These can be followed by repeat imaging as an outpatient.. [**First Name8 (NamePattern2) **]   [**First Name4 (NamePattern1) 1775**]   [**Last Name (NamePattern1) **] , M.D.. [**MD Number(1) 1776**]. Dictated By:. [**Hospital 1807**]  MEDQUIST36. D:   [**2151-8-5**]   12:11. T:   [**2151-8-5**]   12:21. JOB#: [**Job Number 1808**]'

In [67]:
show_medical_abbreviation(nlp, note_str)

/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:283: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


[]

In [68]:
abbreviation_note_str = "Spinal and bulbar muscular atrophy (SBMA) is an \
           inherited motor neuron disease caused by the expansion \
           of a polyglutamine tract within the androgen receptor (AR). \
           SBMA can be caused by this easily."

show_medical_abbreviation(nlp, abbreviation_note_str)

['AR  androgen receptor', 'SBMA  Spinal and bulbar muscular atrophy']

#### Entity Linker

The EntityLinker is a SpaCy component which performs linking to a knowledge base. The linker simply performs a string overlap - based search (char-3grams) on named entities, comparing them with the concepts in a knowledge base using an approximate nearest neighbours search.

Note that entities generally get resolved to a list of different entities. This [blog post](http://sujitpal.blogspot.com/2020/08/disambiguating-scispacy-umls-entities.html) describes one potential way to disambiguate this by figuring out "most likely" set of entities. 

In [17]:
from scispacy.umls_linking import UmlsEntityLinker
# import scispacy.rxnorm_linking

In [18]:
def unified_medical_language_entity_linker(nlp,document):
    """ 
    This function links named entities to the Unified Medical Language System UMLS (https://www.nlm.nih.gov/research/umls/)

    Parameters: 
         model(module): A pretrained biomedical model from ScispaCy(https://allenai.github.io/scispacy/)
         document(str): Document to be processed

    Returns: Attributes of Named entities accessible in the Unified Medical Language System database
     """
    # nlp = model.load()
    linker = UmlsEntityLinker(k=10, max_entities_per_mention=2, \
                              resolve_abbreviations=True)    #parameters are tunable
    nlp.add_pipe(linker)
    doc = nlp(document)
    entity = doc.ents
    entity = [str(item) for item in entity]               # convert each entity tuple to list of strings
    entity = str(OrderedDict.fromkeys(entity))            # returns unique entities only
    entity = nlp(entity).ents                             # convert unique entities back to '.ents' object
    # for entity in entity:
    #     for umls_ent in entity._.umls_ents:
    #         print("Entity Name:" ,entity)
    #         Concept_Id, Score = umls_ent
    #         print('Concept_Id = {} Score = {}'.format(Concept_Id,Score))
    #         print(linker.umls.cui_to_entity[umls_ent[0]])
    return doc, entity, linker

In [19]:
doc, entity, linker = unified_medical_language_entity_linker(nlp, note_str)

https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/umls/tfidf_vectors_sparse.npz not found in cache, downloading to /tmp/tmpu_f25k_1
Finished download, copying /tmp/tmpu_f25k_1 to cache at /home/yutsumi/.scispacy/datasets/b7aadedb8a0dc19e86aca4e866e8767a419889c39f92f96e9458ee264bc13783.d670f5df5e7ad62f30ba05df2f48cafc6058117f2e352385f289872ac8c95028.tfidf_vectors_sparse.npz
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/umls/nmslib_index.bin not found in cache, downloading to /tmp/tmpk09jp7qx
Finished download, copying /tmp/tmpk09jp7qx to cache at /home/yutsumi/.scispacy/datasets/dda67f3ef5951cfedf2de7de7b3561d45ce9e8ed92346cc196310a76b15fe0b6.fdebf628ed43f2d0fba4f250d97a57ddcb36b183308702481009f67c5d771465.nmslib_index.bin
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/umls/tfidf_vectorizer.joblib not found in cache, downloading to /tmp/tmp3tpv99sz
Finished download, copying /tmp/tmp3tpv99sz to cache at /home/yutsumi/.scispacy/datasets/2d16de7bdaeca09492930b4065

/opt/conda/envs/opennotes/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/envs/opennotes/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/umls/concept_aliases.json not found in cache, downloading to /tmp/tmpmdxiqxce
Finished download, copying /tmp/tmpmdxiqxce to cache at /home/yutsumi/.scispacy/datasets/d485a39692e39f93339abf6102c3336dda07d84d8ae46a03d55079b54cad1137.a65425f2cddfa1f741a17c18fc68797f5434547425e30bbe236dea0dcb7a9389.concept_aliases.json
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/umls_2020_aa_cat0129.jsonl not found in cache, downloading to /tmp/tmphe42vzv6
Finished download, copying /tmp/tmphe42vzv6 to cache at /home/yutsumi/.scispacy/datasets/cf6297e16bc34e731db62e18b99603d440223279d6574e101d610ab5c713b265.1b0c2eaa35a3a31246741feac4b0255581214437fb86cd941018283668d461b0.umls_2020_aa_cat0129.jsonl
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/umls_semantic_type_tree.tsv not found in cache, downloading to /tmp/tmp7srh4sl2
Finished download, copying /tmp/tmp7srh4sl2 to cache at /home/yutsumi/.scispacy/datasets/21a1012c532c3a431d60

/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:283: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [20]:
entity = doc.ents
entity = [str(item) for item in entity]               # convert each entity tuple to list of strings
entity = str(OrderedDict.fromkeys(entity))            # returns unique entities only
entity = nlp(entity).ents                             # convert unique entities back to '.ents' object

In [21]:
entity[1]._.umls_ents

[('C0034599', 1.0), ('C0205483', 1.0)]

In [22]:
for entity in entity:
    for umls_ent in entity._.umls_ents:
        print("Entity Name:" ,entity)
        Concept_Id, Score = umls_ent
        print('Concept_Id = {} Score = {}'.format(Concept_Id,Score))
        print(linker.umls.cui_to_entity[umls_ent[0]])


Entity Name: RADIOLOGIC
Concept_Id = C0034599 Score = 1.0
CUI: C0034599, Name: Radiology Specialty
Definition: A specialty concerned with the use of x-ray and other forms of radiant energy in the diagnosis and treatment of disease.
TUI(s): T091
Aliases (abbreviated, total: 14): 
	 radiology specialty, Radiology - specialty, Radiology, Radiology, Radiology, Radiology, Radiology, radiology, radiology, radiology
Entity Name: RADIOLOGIC
Concept_Id = C0205483 Score = 1.0
CUI: C0205483, Name: Radiologic
Definition: None
TUI(s): T169
Aliases: (total: 5): 
	 Radiologic, radiologic, Roentgenologic, roentgenologic, Radiologic (qualifier value)
Entity Name: STUDIES
Concept_Id = C0947630 Score = 1.0
CUI: C0947630, Name: Scientific Study
Definition: None
TUI(s): T059
Aliases: (total: 7): 
	 scientific study, scientific studies, Study, Study, Study, study, studies
Entity Name: STUDIES
Concept_Id = C0085973 Score = 0.8524726629257202
CUI: C0085973, Name: Case Study
Definition: A detailed report of th

In [23]:
displacy_image = displacy.render(doc, jupyter=True,style='ent')

Other entity recognition tools
* [Med7](https://kormilitzin.medium.com/med7-clinical-information-extraction-system-in-python-and-spacy-5e6f68ab1c68) 

[Github](https://github.com/kormilitzin/med7)

"In order to maximise the utilisation of free-text electronic health records (EHR), we focused on a particular subtask of clinical information extraction and developed a dedicated named-entity recognition model Med7 for identification of 7 medication-related concepts, dosage, drug names, duration, form, frequency, route of administration and strength. The model is trained on MIMIC-III, which is one of the largest openly available dataset developed by the MIT Lab for Computational Physiology."

In [ ]:
!pip install https://www.dropbox.com/s/xbgsy6tyctvrqz3/en_core_med7_lg.tar.gz?dl=1

     |██████████████████████████████  | 834.1MB 1.9MB/s eta 0:00:31tcmalloc: large alloc 1147494400 bytes == 0x56296ac38000 @  0x7fa426c7b615 0x562931faf06c 0x56293208eeba 0x562931fb1e8d 0x5629320a399d 0x562932025fe9 0x562932020b0e 0x562931fb377a 0x562932025e50 0x562932020b0e 0x562931fb377a 0x56293202286a 0x5629320a47c6 0x562932021ee2 0x5629320a47c6 0x562932021ee2 0x5629320a47c6 0x562932021ee2 0x5629320a47c6 0x562932126431 0x562932087049 0x562931ff1c84 0x562931fb28e9 0x562932026ade 0x562931fb369a 0x562932021a45 0x562932020e0d 0x562931fb377a 0x562932021a45 0x562931fb369a 0x562932021a45
     |████████████████████████████████| 892.8MB 16kB/s 
  Using cached https://files.pythonhosted.org/packages/1b/d8/0361bbaf7a1ff56b44dca04dace54c82d63dad7475b7d25ea1baefafafb2/spacy-3.0.6-cp37-cp37m-manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/82/a5/b5021c74c04cac35a27d34cbf3146d86eb8e173b4491888bc4908c4c8b3b/catalogue-2.0.3-py3-none-any.whl
  Using cached https://file

In [ ]:
import spacy

med7 = spacy.load("en_core_med7_lg")

# create distinct colours for labels
col_dict = {}
seven_colours = ['#e6194B', '#3cb44b', '#ffe119', '#ffd8b1', '#f58231', '#f032e6', '#42d4f4']
for label, colour in zip(med7.pipe_labels['ner'], seven_colours):
    col_dict[label] = colour

options = {'ents': med7.pipe_labels['ner'], 'colors':col_dict}

text = 'A patient was prescribed Magnesium hydroxide 400mg/5ml suspension PO of total 30ml bid for the next 5 days.'
doc = med7(text)

spacy.displacy.render(doc, style='ent', jupyter=True, options=options)

[(ent.text, ent.label_) for ent in doc.ents]

OSError: ignored

### Experiment on MIMIC

More info on `UmlsEntityLinker` ([source code](https://github.com/allenai/scispacy/blob/4ade4ec897fa48c2ecf3187caa08a949920d126d/scispacy/linking.py#L9))

`nlp` is a `Language` (?) object that returns a `Doc` ([source code](https://github.com/explosion/spaCy/blob/master/spacy/tokens/doc.pyx#L104)), which is a sequence of `Token` word objects ([source code](https://github.com/explosion/spaCy/blob/master/spacy/tokens/token.pyx#L26)).

https://spacy.io/api/doc

https://spacy.io/api/token

#### Default vectorization of sentences and words

We start off by choosing a model [here](https://allenai.github.io/scispacy/), e.g. `en_core_sci_sm`. The model we choose is tuned for biomedical vocabulary and has a large collection of word vectors good for our vocabulary. 

When we pass a sentence through the model, e.g. `nlp`, each word in the sentence gets vectorized. These word vecs can be accessed through the output `Doc` instance. This instance also contains sentence vecs, which are just averaged across the word vecs. 

#### From word vectorization to entity recognition

Once words are vectorized, scispacy runs KNN in the background to map words to the entity labels specified by our linkers. 

#### Measuring similarity 

We can leverage the vectorizations to measure similarities. For example, we can use the function `Doc.similarity` ([source code](https://spacy.io/api/doc#similarity)) to measure similarity between different sentences. Their default uses cosine similarity using average of word vectors as their sentence vectorization. 

Note that we can also measure similarity between other instances (e.g. `Token`), or between two different instances (e.g. between `Token` and `Doc`). 

In [115]:
nlp = spacy.load("en_core_sci_sm")

In [174]:
from scispacy.umls_linking import UmlsEntityLinker

In [175]:
# abbreviation_pipe = AbbreviationDetector(nlp) # automatically included with UMLS linker
# nlp.add_pipe(abbreviation_pipe)
linker = UmlsEntityLinker(k=10,                          # number of nearest neighbors to look up from
                          threshold=0.7,                 # confidence threshold to be added as candidate
                          max_entities_per_mention=1,    # number of entities returned per concept (todo: tune)
                          filter_for_definitions=False,  # no definition is OK
                          resolve_abbreviations=True)    # resolve abbreviations before linking
# linker = EntityLinker(k=10,                          # number of nearest neighbors to look up from
#                       threshold=0.7,                 # confidence threshold to be added as candidate
#                       max_entities_per_mention=1,    # number of entities returned per concept (todo: tune)
#                       resolve_abbreviations=True,    # resolve abbreviations before linking
#                       linker_name='umls')
nlp.add_pipe(linker)

/opt/conda/envs/opennotes/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/envs/opennotes/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [224]:
SEMANTIC_TYPE_TO_NAME

{'T047': 'Disease or Syndrome',
 'T121': 'Pharmacologic Substance',
 'T023': 'Body Part, Organ, or Organ Component',
 'T061': 'Therapeutic or Preventive Procedure',
 'T060': 'Diagnostic Procedure',
 'T059': 'Laboratory Procedure'}

In [176]:
class SentRep(object):
    def __init__(self, sentence, nlp, linker, filter_map=None):
        """ 
        Given a sentence and scispacy model with UMLS linker, 
        SentRep extracts important information and stores them as attributes. 
        
        Set filter_map to be map of semantic type identifiers to name (e.g. SEMANTIC_TYPE_TO_NAME)
        to only extract those attributes.
        """
        self.umls_cui_map = linker.umls.cui_to_entity # maps CUI to entity information
        self.filter_map   = filter_map
        self.is_filter    = (filter_map is not None)
        
        self.doc = nlp(sentence)
        
        self.semantic_types = []
        self.semantic_names = []  # names of categories of entities
        self.canonical_names = [] # names of types of entities
        self.get_umls_info()

    def get_umls_info(self):
        for ent in self.doc.ents: # extract info (umls) for each entity
            cui, _ = ent._.umls_ents[0] # assuming `max_entites_per_mention=1` for now
            cui_info = self.umls_cui_map[cui]
            
            ent_valid_type_list = [t in self.filter_map for t in cui_info.types]
            ent_valid_type = any(ent_valid_type_list) # checks if entity is a valid type
            
            if not self.is_filter or ent_valid_type: # only add to list if we're not filtering of it's valid
                self.canonical_names.append(cui_info.canonical_name)
                for (stype, keep) in zip(cui_info.types, ent_valid_type_list):
                    if keep:
                        self.semantic_types.append(stype)
                        self.semantic_names.append(self.filter_map[stype])
            
        self.semantic_types = set(self.semantic_types)
        self.semantic_names = set(self.semantic_names)
        self.canonical_names = set(self.canonical_names)

In [177]:
srep = SentRep(sent, nlp, linker, filter_map=SEMANTIC_TYPE_TO_NAME)

/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:283: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [178]:
srep.semantic_types, srep.semantic_names, srep.canonical_names

({'T121'}, {'Pharmacologic Substance'}, {'Solu-Medrol'})

In [179]:
SEMANTIC_TYPE_TO_NAME

{'T047': 'Disease or Syndrome',
 'T121': 'Pharmacologic Substance',
 'T023': 'Body Part, Organ, or Organ Component',
 'T061': 'Therapeutic or Preventive Procedure',
 'T060': 'Diagnostic Procedure',
 'T059': 'Laboratory Procedure'}

In [182]:
len(all_sent_strs)

1137

In [201]:
all_sreps = []
all_srep_canon_names = []
for sent in all_sent_strs:
    try:
        srep = SentRep(sent, nlp, linker, filter_map=SEMANTIC_TYPE_TO_NAME)
        if len(srep.canonical_names) > 0:
            all_sreps.append(srep)
            all_srep_canon_names.append(list(srep.canonical_names))
    except:
        continue

/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:283: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [202]:
len(all_sreps)

293

In [210]:
corpus = list(map(lambda x: ' '.join(x), all_srep_canon_names))
corpus[:10]

['amlodipine',
 'Operative Surgical Procedures Invasive procedure',
 'Esophagogastroduodenoscopy Intubation',
 'oxygen COPD pharmacologic substance Congestive heart failure',
 'COPD pharmacologic substance',
 'Levaquin',
 'Lung consolidation Plain chest X-ray',
 'Electrocardiography',
 'Leukocytosis Lactic acid measurement',
 'Intubation']

In [211]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = CountVectorizer()
corpus = list(map(lambda x: ' '.join(x), all_srep_canon_names))
X = vectorizer.fit_transform(corpus)
X = X.toarray()

In [214]:
sim_X = cosine_similarity(X)
sim_X

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [216]:
(sim_X>0).sum()

3571

In [218]:
simx, simy = np.where(sim_X>0)

In [223]:
for x, y in zip(simx, simy):
    if x!=y:
        print(f"S1: {all_sreps[x].doc}")
        print(f"S2: {all_sreps[y].doc}")
        print(f"S1 canonical concepts: {all_srep_canon_names[x]}")
        print(f"S2 canonical concepts: {all_srep_canon_names[y]}")
        print("**********************************")

S1: Allergies: Amlodipine
S2: Allergies: Amlodipine
S1 canonical concepts: ['amlodipine']
S2 canonical concepts: ['amlodipine']
**********************************
S1: Major Surgical or Invasive Procedure:
S2: [**Last Name (STitle) 1407**]  was very involved in discussions regarding patient's code status.
S1 canonical concepts: ['Operative Surgical Procedures', 'Invasive procedure']
S2 canonical concepts: ['Discussion (procedure)']
**********************************
S1: Major Surgical or Invasive Procedure:
S2: The patient's family felt that she would not want a tracheostomy.
S1 canonical concepts: ['Operative Surgical Procedures', 'Invasive procedure']
S2 canonical concepts: ['Tracheostomy procedure']
**********************************
S1: Major Surgical or Invasive Procedure:
S2: A tube was placed into your stomach and you underwent an endoscopic procedure to look at your esophagus and stomach.
S1 canonical concepts: ['Operative Surgical Procedures', 'Invasive procedure']
S2 canonical

#### What to do from here

* group sentences by semantic type and map them to each category of conflicts to check for (e.g. T047 --> disease and symptom errors)
* use canonical names to group sentences with similar or dissimilar topics together (note: once we incorporate RxNorm and Med7, include concepts from those into canonical names as well)

In [22]:
doc = nlp(note_str) 
# doc = nlp(abbreviation_note_str)
displacy_image = displacy.render(doc, jupyter=True, style='ent')

/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:283: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
/opt/conda/envs/opennotes/lib/python3.7/site-packages/scispacy/candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


In [70]:
entity = doc.ents
entity = [str(item) for item in entity]               # convert each entity tuple to list of strings
entity = str(OrderedDict.fromkeys(entity))            # returns unique entities only
entity = nlp(entity).ents                             # convert unique entities back to '.ents' object

In [71]:
# Extracting info from "ents" object
ent = entity[1]

# 1. Name of entity
print(ent)

# 2. UMLS mapping
umls_ents = ent._.umls_ents
print(umls_ents)

# 3. CUI, confidence
umls_ent = umls_ents[0]
cui, confidence = umls_ent
print(f"CUI: {cui}, Confidence: {confidence}")

# 4. CUI information
ent_cui_info = linker.umls.cui_to_entity[cui]
print(ent_cui_info)

bulbar muscular atrophy
[('C1839259', 0.9103652834892273), ('C0026846', 0.800467312335968)]
CUI: C1839259, Confidence: 0.9103652834892273
CUI: C1839259, Name: Bulbo-Spinal Atrophy, X-Linked
Definition: An X-linked recessive form of spinal muscular atrophy. It is due to a mutation of the gene encoding the ANDROGEN RECEPTOR.
TUI(s): T047
Aliases (abbreviated, total: 44): 
	 Bulbo-Spinal Atrophy, X-Linked, Atrophies, X-Linked Bulbo-Spinal, Bulbo Spinal Atrophy, X Linked, Bulbo-Spinal Atrophies, X-Linked, X-Linked Bulbo-Spinal Atrophies, Atrophy, X-Linked Bulbo-Spinal, X Linked Bulbo Spinal Atrophy, X-Linked Bulbo-Spinal Atrophy, Bulbospinal muscular atrophy, Kennedy Syndrome


In [79]:
print(linker.umls.cui_to_entity[umls_ents[0][0]].canonical_name)
print(linker.umls.cui_to_entity[umls_ents[1][0]].canonical_name)

Bulbo-Spinal Atrophy, X-Linked
Muscular Atrophy


#### Accessing info

In [80]:
# ent_cui_info.aliases
# ent_cui_info.canonical_name
# ent_cui_info.concept_id
# ent_cui_info.count
# ent_cui_info.definition
# ent_cui_info.index
ent_cui_info.types # https://www.nlm.nih.gov/research/umls/sourcereleasedocs/current/PSY/stats.html

['T047']

`...types` refers to the broader overall category that this CUI belongs to. We could use this to filter out types of interest for our use cases. 

Link to table: https://www.nlm.nih.gov/research/umls/sourcereleasedocs/current/LCH/stats.html

* T047 - Disease or Syndrome
* T121 - Pharmacologic Substance
* T023 - Body Part, Organ, or Organ Component
* T061 - Therapeutic or Preventive Procedure 
* T060 - Diagnostic Procedure
* T059 - Laboratory Procedure

In [104]:
SEMANTIC_TYPES = ['T047', 'T121', 'T023', 'T061', 'T060', 'T059']
SEMANTIC_NAMES = ['Disease or Syndrome', 'Pharmacologic Substance', 'Body Part, Organ, or Organ Component', \
                  'Therapeutic or Preventive Procedure', 'Diagnostic Procedure', 'Laboratory Procedure']
SEMANTIC_TYPE_TO_NAME = dict(zip(SEMANTIC_TYPES, SEMANTIC_NAMES))

https://medium.com/@oyewusiwuraola/how-to-use-scispacy-for-biomedical-named-entity-recognition-abbreviation-resolution-and-link-umls-87d3f7c08db2

# Other: Relevant papers

## [MedNLI](https://www.aclweb.org/anthology/D18-1187.pdf)



In [ ]:
!wget -r -N -c -np --user yutsumi --ask-password https://physionet.org/files/mednli/1.0.0/

In [ ]:
!git clone https://github.com/jgc128/mednli.git

In [ ]:
!pip install -r mednli/requirements.txt

In [ ]:
!pip install sacred

     |████████████████████████████████| 112kB 4.3MB/s eta 0:00:01
     |████████████████████████████████| 102kB 3.8MB/s 
     |████████████████████████████████| 163kB 7.6MB/s 
     |████████████████████████████████| 71kB 5.3MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-cp37-none-any.whl size=22245 sha256=fa0ca522388f4c7e3a73ff2256acdbbac77381ab6c0b9fa307a24d2c504fcb6e
  Stored in directory: /root/.cache/pip/wheels/2e/15/f5/aa2a056d223903b52cf4870134e3a01df0c723816835dd08db
Successfully built py-cpuinfo


In [ ]:
!mkdir mednli/data

mkdir: cannot create directory ‘mednli/data’: No such file or directory


In [ ]:
!mv physionet.org/files/mednli/1.0.0 mednli/data

mv: cannot stat 'physionet.org/files/mednli/1.0.0': No such file or directory


In [ ]:
import json
import pandas as pd

In [ ]:
filename = 'mednli/data/1.0.0/mli_dev_v1.jsonl'

all_rows = []
with open(str(filename)) as f:
    for i, line in enumerate(f):
        row = json.loads(line)
        if row['gold_label'] != '-':
            all_rows.append(row)

nli_data = pd.DataFrame(all_rows)

FileNotFoundError: ignored

In [ ]:
nli_sub_data = nli_data[['sentence1', 'sentence2', 'gold_label']]

nli_sub_data.loc[nli_sub_data.gold_label == 'contradiction']

In [ ]:
print(f"*****************************")
for _, row in nli_sub_data.loc[nli_sub_data.gold_label == 'contradiction'].iterrows():
  print(f"Sentence 1: {row.sentence1}")
  print(f"Sentence 2: {row.sentence2}")
  print(f"*****************************")